In [ ]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Deploying a Kubeflow Pipeline on Google Cloud Platform (GCP)

## Choosing an installation option

1. Do you want to use other Kubeflow components in addition to Pipelines?

    If yes, choose the [full Kubeflow deployment](https://www.kubeflow.org/docs/gke/deploy/).
1. Do you want to use Kubeflow Pipelines with [multi-user support](https://github.com/kubeflow/pipelines/issues/1223)?

    If yes, choose the [full Kubeflow deployment](https://www.kubeflow.org/docs/gke/deploy//) with version >= v1.1.
1. Do you deploy on Google Cloud?

    If yes, deploy [Kubeflow Pipelines Standalone](https://www.kubeflow.org/docs/pipelines/installation/standalone-deployment). You can also
    use [Google Cloud AI Platform Pipelines](https://cloud.google.com/ai-platform/pipelines/docs/setting-up) to deploy Kubeflow Pipelines
    using a user interface, but there are limitations in
    customizability and upgradability. For details, please read corresponding
    sections.
1. You deploy on other platforms.

    Please compare your platform specific [full Kubeflow](https://www.kubeflow.org/docs/gke/deploy/) with the
    [Kubeflow Pipelines Standalone](https://www.kubeflow.org/docs/pipelines/installation/standalone-deployment) before making your decision.

**In the following, we will demonstrate [Standalone deployment](https://www.kubeflow.org/docs/pipelines/installation/standalone-deployment/) on GCP**

To deploy Kubeflow Pipelines, you must be granted the `Viewer (roles/viewer)` and `Kubernetes Engine Admin (roles/container.admin)` roles on the project. The `Viewer` role provides read access to all resources within a project. The `Kubernetes Engine Admin` role provides full access to Kubernetes clusters and Kubernetes API objects.

## Set up environment variables
Set up environment variables to use while installing Kubeflow. Replace variable placeholders (for example, `<VARIABLE NAME>`) with the correct values for your environment.

In [ ]:
# Set your GCP project ID and the zone where you want to create the Kubeflow deployment
%env PROJECT=kubeflow-pipeline-fantasy
%env ZONE=asia-southeast1-c

%env CLUSTER_NAME=deploy-kfp

%env MACHINE_TYPE=n1-standard-2

# Warning: Using SCOPES="cloud-platform" grants all GCP permissions to the cluster. 
# For a more secure cluster setup, refer to Authenticating Pipelines to GCP.
%env SCOPES=cloud-platform

%env PIPELINE_VERSION=1.0.3

If you have an existing Kubernetes cluster, skip the below cell. For more detail, see the GKE guide to [creating a cluster](https://cloud.google.com/kubernetes-engine/docs/how-to/creating-a-cluster) for Google Cloud Platform (GCP).

In [ ]:
! gcloud container clusters create ${CLUSTER_NAME} \
     --zone ${ZONE} \
     --machine-type ${MACHINE_TYPE} \
     --scopes ${SCOPES}

## Deploying Kubeflow Pipelines

In [ ]:
! kubectl apply -k "github.com/kubeflow/pipelines/manifests/kustomize/cluster-scoped-resources?ref=$PIPELINE_VERSION"

! kubectl wait --for condition=established --timeout=60s crd/applications.app.k8s.io

! kubectl apply -k "github.com/kubeflow/pipelines/manifests/kustomize/env/dev?ref=$PIPELINE_VERSION"

## Get the public URL for the Kubeflow Pipelines UI

In [ ]:
! kubectl describe configmap inverse-proxy-config -n kubeflow | grep googleusercontent.com

You may also access the [pipelines dashboard](https://console.cloud.google.com/ai-platform/pipelines/clusters) by opening AI Platform Pipelines in the Cloud Console.